### Лабораторная работа #1.
 Реализовать скрапер по seed urls list, и сохранить текст + изображения с 200 страниц (в файлы на pc). Структура: 1 каталог, внутри которого текстовой файл + файлы с изображениями. А также отдельный файл с мета информацией (url, количественные характеристики загруженных данных).

In [48]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_links(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # Извлечение всех ссылок на странице
    links = []
    for a_tag in soup.find_all('a', href=True):
        href = a_tag.get('href')
        absolute_url = urljoin(base_url, href)
        # Проверка, что ссылка заканчивается на '.html' и содержит 'football'
        # 201 потому что это костыль, у меня первая ссылка идёт с подссылкой не 'footaball', а 'mediafootball', а я не хочу этого
        if absolute_url.endswith('.html') and 'football' in absolute_url and len(links) < 201:
            links.append(absolute_url)
        if len(links) >= 201:
            break
    
    return links

link_1 = 'https://www.sports.ru/football/news/'
link_2 = 'https://www.espn.com/soccer/'
link_3 = 'https://www.skysports.com/football'

links_list_sports_ru = get_links(link_1)
links_list_sports_ru = links_list_sports_ru[1:]
print(len(links_list_sports_ru))

200


In [49]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from datetime import datetime

def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def fetch_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

def parse_page(html, base_url):
    soup = BeautifulSoup(html, 'html.parser')
    
    # Извлечение текста
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text(separator=' ')
    text = ' '.join(text.split())
    
    # Извлечение URL изображений
    images = []
    for img in soup.find_all('img'):
        img_url = img.get('src')
        if img_url:
            img_url = urljoin(base_url, img_url)
            images.append(img_url)
    
    return text, images

def append_text(text, path):
    with open(path, 'a', encoding='utf-8') as f:
        f.write(text + '\n\n')
        #print(len(text))

def save_images(images, directory, start_index):
    # for idx, img_url in enumerate(images):
    #     try:
    #         img_data = requests.get(img_url).content
    #         img_name = os.path.join(directory, f'image_{idx + 1}.jpg')
    #         with open(img_name, 'wb') as f:
    #             f.write(img_data)
    #     except requests.RequestException as e:
    #         print(f"Error saving image {img_url}: {e}")
    image_index = start_index
    for img_url in images:
        try:
            img_data = requests.get(img_url).content
            img_name = os.path.join(directory, f'image_{image_index}.jpg')
            with open(img_name, 'wb') as f:
                f.write(img_data)
            image_index += 1
        except requests.RequestException as e:
            print(f"Error saving image {img_url}: {e}")
    return image_index

def scrape(seed_urls, output_directory, max_pages=5):
    create_directory(output_directory)
    
    meta_info = []
    pages_scraped = 0
    image_index = 1
    sum_page_images, sum_len_page_text = 0, 0
    text_file_path = os.path.join(output_directory, 'content.txt')
    
    for url in seed_urls:
        if pages_scraped >= max_pages:
            break

        page_html = fetch_page(url)
        if page_html:
            page_text, page_images = parse_page(page_html, url)
            
            # Сохранение текста и изображений
            append_text(page_text, text_file_path)
            image_index = save_images(page_images, output_directory, image_index)
            
            # Сохранение метаинформации
            sum_len_page_text += len(page_text)
            sum_page_images += len(page_images)
            
            meta_info.append({
                'url': url,
                'text_length': len(page_text),
                'num_images': len(page_images)
            })
            
            pages_scraped += 1
    
    # Запись метаинформации в файл
    with open(os.path.join('scraped_data', 'meta_info_www_sports_ru.txt'), 'w', encoding='utf-8') as f:
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"{current_time}\n")
        f.write(f"Sum Text Length: {sum_len_page_text}\n")
        f.write(f"Sum Number of Images: {sum_page_images}\n\n")
        for info in meta_info:
            f.write(f"URL: {info['url']}\n")
            f.write(f"Text Length: {info['text_length']}\n")
            f.write(f"Number of Images: {info['num_images']}\n\n")
        


seed_urls = links_list_sports_ru

output_directory = 'scraped_data/www_sports_ru'
scrape(seed_urls, output_directory, max_pages=200)